## Setup

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from IPython.display import Image

In [2]:
import os

In [3]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
!conda install tensorflow -y

In [ ]:
!conda install cv2 -y

In [ ]:
# save file paths for each set of images
trainN = 'data/train/NORMAL'
trainP = 'data/train/PNEUMONIA'
testN = 'data/test/NORMAL'
testP = 'data/test/PNEUMONIA'
valN = 'data/val/NORMAL'
valP = 'data/val/PNEUMONIA'

In [ ]:
# count of images
numTrainN = len(os.listdir(trainN))
numTrainP = len(os.listdir(trainP))
numTestN = len(os.listdir(testN))
numTestP = len(os.listdir(testP))
numValN = len(os.listdir(valN))
numValP = len(os.listdir(valP))

print('Total training normal lung images: ', numTrainN)
print('Total training pneumonia lung images: ', numTrainP)
print('Total test normal lung images: ', numTestN)
print('Total test pneumonia lung images: ', numTestP)
print('Total validation normal lung images: ', numValN)
print('Total validation pneumonia lung images: ', numValP)

### Show images

In [ ]:
image = Image(filename='data/train/NORMAL/IM-0115-0001.jpeg')

In [ ]:
display(image)

## Prepare Data

In [ ]:
#pre-processing functions
def image_to_feature_vector(image, size=(32,32)):
    # resize image to fixed size & flatten into list of pixel intensities
    return cv.resize(image,size).flatten()

def extract_color_histogram(image, bins=(8, 8, 8)):
    # extract a 3D color histogram using the supplied number of bins per channel
    hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    hist = cv.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
    
    # handle normalizing the histogram 
    cv.normalize(hist, hist)
    
    # return the flattened histogram as the feature vector
    return hist.flatten()

In [ ]:
data = 